## 0)import libraries 

In [38]:
"""
Thesis_PhD_Qianqian Predicted_fluxes_glboal_stripes
date: 24-Feb-2023
author: Qianqian
Contact: q.han@utwente.nl
-------------------------------------
Description: 
"""
# libraries
import os
import joblib
from osgeo import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from rasterio.warp import reproject, Resampling
import xarray as xr
import pickle
from PyStemmusScope import variable_conversion as vc
from rasterio.warp import reproject, Resampling
from sklearn.preprocessing import OneHotEncoder
import time
import re
import glob
import rioxarray
import dask.array
from dask.distributed import Client, progress

## 1)settings for Dask

In [39]:
# Set the MALLOC_TRIM_THRESHOLD_ environment variable
os.environ['MALLOC_TRIM_THRESHOLD_'] = '0'

# Run your Dask script or notebook
...

Ellipsis

In [40]:
client = Client(n_workers=8, threads_per_worker=2)
client

/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35349 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/35349/status,
Dashboard: /proxy/35349/status,Workers: 8
Total threads: 16,Total memory: 28.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36487,Workers: 8
Dashboard: /proxy/35349/status,Total threads: 16
Started: Just now,Total memory: 28.00 GiB
Comm: tcp://127.0.0.1:37245,Total threads: 2
Dashboard: /proxy/37169/status,Memory: 3.50 GiB
Nanny: tcp://127.0.0.1:36193,


## 2)define working path, load trained model, define functions

In [41]:
workingPath = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/"

In [42]:
# load trained RF model, better use not parallel model with Dask
with open(workingPath+'3RF_train/RFLEH-LAI-hc-CO2-SSM-Vcmo-IGBP_multi7_1core_snellius0629.pkl', 'rb') as f:
    rfLEHmulti = pickle.load(f)

In [43]:
# function for loading the trained model
def load_model(path):
    # load trained RF model, better use not parallel model with Dask
    with open(path, 'rb') as f:
        rfLEHmulti = pickle.load(f)
    return rfLEHmulti

In [44]:
# function for getting directory of input data
def get_directories_with_number_and_ending(directory_path, ending):
    directories = []
    if ending:
        pattern = re.compile(r'^\d+.*{}$'.format(re.escape(ending)))  # Match directory names with number at the beginning and specific ending
    else:
        pattern = re.compile(r'^\d+$')  # Match directory names with only numbers

    for entry in os.scandir(directory_path):
        if entry.is_dir():
            directory_name = entry.name
            if pattern.match(directory_name):
                directories.append(directory_name)

    return directories

## 3) read data

In [45]:
### define the path of input data, set the reference file for spatial resample
inputData = workingPath+"1input_data/"
# get all the filefolders named as year
year_list = sorted(get_directories_with_number_and_ending(workingPath+"1input_data/", "global"))
# the input data from year[0] to year[..], based on the process id in sbatch script
year = year_list[4]
print(year)

2014global


In [46]:
import geopandas as gpd
from shapely.geometry import box

shapefile_path = "/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/Emulator/input_data/EuropeBoundary.shp"
gdf = gpd.read_file(shapefile_path)
bbox = gdf.total_bounds
bbox

array([-31.28903052,  34.93055094,  68.93136141,  81.85192337])

In [12]:
lat1 = 40
lat2 = 70
lon1 = 2
lon2 = 32

In [48]:
## 0) read era5land data
all1 = xr.open_zarr(inputData+year+"/era5land/era5land2014_10km1030.zarr") 

In [14]:
all1.chunks

Frozen({'time': (500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 260), 'latitude': (250, 219), 'longitude': (250, 250, 250, 250, 2)})

In [51]:
all1

<xarray.Dataset>
Dimensions:    (time: 8760, latitude: 469, longitude: 1002)
Coordinates:
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * longitude  (longitude) float32 -31.2 -31.1 -31.0 -30.9 ... 68.7 68.8 68.9
  * time       (time) datetime64[ns] 2014-01-01 ... 2014-12-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
    strd       (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(500, 250, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 03:39:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [15]:
chunksizeLat = 110
chunksizeLon = 110

In [16]:

all1 = all1.chunk({"time":500, "latitude":chunksizeLat, "longitude":chunksizeLon}) #
all1 = all1.sel(
    latitude=slice(lat2, lat1),  
    longitude=slice(lon1, lon2)  
)

In [59]:
all1.chunks

Frozen({'time': (500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 260), 'latitude': (11,), 'longitude': (8, 13)})

In [54]:
all_resample = all1.resample(time="1800S").interpolate('linear')
all_resample = all_resample.isel(time=slice(0,17000))


In [55]:
all_resample

<xarray.Dataset>
Dimensions:    (time: 17519, latitude: 469, longitude: 1002)
Coordinates:
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * longitude  (longitude) float32 -31.2 -31.1 -31.0 -30.9 ... 68.7 68.8 68.9
  * time       (time) datetime64[ns] 2014-01-01 ... 2014-12-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
    ssrd       (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
    strd       (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(17519, 250, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-19 03:39:36 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [17]:
all_resample = all_resample.persist()

In [17]:
all_resample.chunks

Frozen({'time': (17000,), 'latitude': (102, 110, 89), 'longitude': (108, 110, 83)})

In [50]:
%%time
# calculate the Rin and Rli difference for every hour
Rin = all1['ssrd'].diff("time")/3600  #xr.concat([all1['ssrd'].isel(time=0),all1['ssrd']], dim="time")
Rin[0::24] = all1['ssrd'][1::24]/3600 # assign the original values in t01
# Rin.mean(dim=['longitude','latitude']).compute()[48]
Rli = all1['strd'].diff("time")/3600  
Rli[0::24] = all1['strd'][1::24]/3600
# Rli.mean(dim=['longitude','latitude']).plot()
Rin = Rin.resample(time="1800S").interpolate('linear')
Rli = Rli.resample(time="1800S").interpolate('linear')
Rin

CPU times: user 409 ms, sys: 6.37 ms, total: 415 ms
Wall time: 412 ms


/home/qiahan/.local/lib/python3.9/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 20 entries) hit.
Flushing oldest 4 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


<xarray.DataArray 'ssrd' (time: 17517, latitude: 469, longitude: 1002)>
dask.array<transpose, shape=(17517, 469, 1002), dtype=float32, chunksize=(17517, 250, 250), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * longitude  (longitude) float32 -31.2 -31.1 -31.0 -30.9 ... 68.7 68.8 68.9
  * time       (time) datetime64[ns] 2014-01-01T01:00:00 ... 2014-12-31T23:00:00

In [19]:
Rin.chunks

((17517,), (102, 110, 89), (108, 110, 83))

In [20]:
all1.nbytes/2**30

23.653120554983616

In [21]:
!ncinfo $inputData/2015global/lai/LAI_global2015_10km.nc

Traceback (most recent call last):
  File "/home/qiahan/.local/bin/ncinfo", line 8, in <module>
    sys.exit(ncinfo())
  File "/home/qiahan/.local/lib/python3.8/site-packages/netCDF4/utils.py", line 551, in ncinfo
    f = Dataset(filename)
  File "src/netCDF4/_netCDF4.pyx", line 2449, in netCDF4._netCDF4.Dataset.__init__
  File "src/netCDF4/_netCDF4.pyx", line 2012, in netCDF4._netCDF4._ensure_nc_success
FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/1input_data//2015global/lai/LAI_global2015_10km.nc'


In [26]:
### 1) read LAI data
lai = xr.open_dataset(inputData+"/2014global/lai/LAI_europe2014_10km.nc",chunks={"latitude":chunksizeLat, "longitude":chunksizeLon})['LAI']

In [27]:
lai = lai.sel(
    latitude=all1.latitude,  
    longitude=all1.longitude,
    method='nearest', tolerance=0.01
)
# lai = lai.chunk({"latitude":chunksizeLat, "longitude":chunksizeLon}) #

In [28]:
lai = lai.assign_coords(longitude=all1.longitude, latitude=all1.latitude)

In [64]:
lai.chunks

((36,), (11,), (8, 13))

In [65]:
lai = lai.resample(time="1800S").interpolate('linear')

In [66]:
lai.chunks

((17041,), (11,), (8, 13))

In [28]:
### 2) read canopy height data
hc_path = inputData+year+'/canopy_height/canopy_height_11kmEurope20230921_10km.nc'
hc = xr.open_dataset(hc_path,chunks={"y":chunksizeLat, "x":chunksizeLon}).rename({'x':'longitude','y':'latitude'})['__xarray_dataarray_variable__']

In [29]:
hc = hc.sel(
    latitude=all1.latitude,  
    longitude=all1.longitude,
    method='nearest', tolerance=0.01
)
# hc = hc.chunk({"latitude":chunksizeLat, "longitude":chunksizeLon}) #
hc = hc.assign_coords(longitude=all1.longitude, latitude=all1.latitude)
hc

<xarray.DataArray '__xarray_dataarray_variable__' (band: 1, latitude: 301,
                                                   longitude: 301)>
dask.array<getitem, shape=(1, 301, 301), dtype=float32, chunksize=(1, 110, 110), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 31.7 31.8 31.9 32.0
  * latitude   (latitude) float32 70.0 69.9 69.8 69.7 ... 40.3 40.2 40.1 40.0
  * band       (band) int64 1
Attributes:
    scales:         1.0
    offsets:        0.0
    descriptions:   b1
    AREA_OR_POINT:  Area
    grid_mapping:   spatial_ref

In [30]:
hc.chunks

((1,), (102, 110, 89), (108, 110, 83))

In [31]:
### 3) read CO2 data
## resampled CO2
ds_co2_10km = xr.open_dataset(inputData+year+"/co2/CAMS_CO2_2014_10km.nc",chunks={"latitude":chunksizeLat, "longitude":chunksizeLon})['co2']
ds_co2_10km = ds_co2_10km.sel(
    latitude=all1.latitude,  
    longitude=all1.longitude,
    method='nearest', tolerance=0.01
)
# ds_co2_10km = ds_co2_10km.chunk({"latitude":chunksizeLat, "longitude":chunksizeLon}) #
ds_co2_10km = ds_co2_10km.assign_coords(longitude=all1.longitude, latitude=all1.latitude)

In [32]:
ds_co2_10km = ds_co2_10km.resample(time="1800S").interpolate('linear')

In [33]:
ds_co2_10km.chunks

((17515,), (102, 110, 89), (108, 110, 83))

In [34]:
### 4) read the resampled SSM data
## netcdf file works well
path_SSM = glob.glob(inputData+year+"/ssm/SM2014Europe11kmEurope20231030_10km.nc")[0]
ds_SSM = xr.open_dataset(path_SSM,chunks={"latitude":chunksizeLat, "longitude":chunksizeLon})['__xarray_dataarray_variable__'].rename('SSM')
# ds_SSM = ds_SSM.sortby(["longitude", "latitude"])
ds_SSM = ds_SSM.sel(
    latitude=all1.latitude,  
    longitude=all1.longitude,
    method='nearest', tolerance=0.01
)
# ds_SSM = ds_SSM.chunk({"latitude":chunksizeLat, "longitude":chunksizeLon}) #
ds_SSM = ds_SSM.assign_coords(longitude=all1.longitude, latitude=all1.latitude)
SSM = ds_SSM.resample(time="1800S").interpolate('linear')/1000 

In [35]:
SSM.chunks

((17473,), (102, 110, 89), (108, 110, 83))

In [36]:
### 5) read Vcmax data
ds_Vcmo = xr.open_dataset(inputData+year+"/vcmax/TROPOMI_Vmax_Tg_mean10km_global.nc",chunks={"y":chunksizeLat, "x":chunksizeLon}).rename({'x':'longitude','y':'latitude'})['__xarray_dataarray_variable__']

In [37]:
ds_Vcmo

<xarray.DataArray '__xarray_dataarray_variable__' (band: 1, latitude: 1801,
                                                   longitude: 3600)>
dask.array<open_dataset-3d5c93dc4ef38b0d46164c12254fb901__xarray_dataarray_variable__, shape=(1, 1801, 3600), dtype=float32, chunksize=(1, 110, 110), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * band       (band) int64 1
Attributes:
    scales:         1.0
    offsets:        0.0
    AREA_OR_POINT:  Area
    grid_mapping:   spatial_ref

In [38]:
ds_Vcmo = ds_Vcmo.sel(
    latitude=all1.latitude,  
    longitude=all1.longitude,
    method='nearest', tolerance=0.01
)
# ds_Vcmo = ds_Vcmo.chunk({"latitude":chunksizeLat, "longitude":chunksizeLon}) #
ds_Vcmo = ds_Vcmo.assign_coords(longitude=all1.longitude, latitude=all1.latitude)

In [39]:
ds_Vcmo.chunks 

((1,), (20, 110, 110, 61), (51, 110, 110, 30))

## why the chunks of ds_Vcmo and landcover after sel function is different from the above variables? because TROPOMI_Vmax_Tg_mean10km_global.nc is global? maybe we should load the data and select the roi, and then chunks?
but we will do the following later, so maybe it is okay like this, only ds_Vcmo and landcover has this problems?

hc=hc.chunk({"latitude":51, "longitude":51}).squeeze()

ds_Vcmo = ds_Vcmo.chunk({"latitude":51, "longitude":51}).squeeze()

landcover = landcover.chunk({"latitude":51, "longitude":51}).squeeze()

In [40]:
### 6) read IGBP data
# read lan
path_landcover = inputData+year+"/igbp/landcover10km_global.nc"
# landcover = xr.open_rasterio(path_landcover, engine="rasterio", chunks="auto").rename({'x':'longitude','y':'latitude'})
landcover = xr.open_dataset(path_landcover, chunks={"y":chunksizeLat, "x":chunksizeLon}).rename({'x':'longitude','y':'latitude'})['lccs_class']
landcover = landcover.sel(
    latitude=all1.latitude,  
    longitude=all1.longitude,
    method='nearest', tolerance=0.01
)
# landcover = landcover.chunk({"latitude":chunksizeLat, "longitude":chunksizeLon}) #
landcover = landcover.assign_coords(longitude=all1.longitude, latitude=all1.latitude)

In [41]:
landcover.chunks

((1,), (20, 110, 110, 61), (51, 110, 110, 30))

In [42]:
# read IGBP unique values
data = pd.read_csv(inputData+'IGBP11unique.csv')['0'].unique()
# read the table for converting landcover to IGBP
IGBP_table = pd.read_csv(inputData+year+"/igbp/lccs_to_igbp_table.csv")

In [43]:
def landcover_to_igbp(landcover, IGBP_table, data):
    IGBP = np.vectorize(IGBP_table.set_index("lccs_class").T.to_dict('records')[0].get)(landcover.values)
    IGBP_all = pd.DataFrame(columns=['IGBP_veg_long1', 'IGBP_veg_long2', 'IGBP_veg_long3','IGBP_veg_long4','IGBP_veg_long5',
                              'IGBP_veg_long6','IGBP_veg_long7','IGBP_veg_long8','IGBP_veg_long9',
                             'IGBP_veg_long10','IGBP_veg_long11'])
    # define one hot encoding for IGBP
    encoder = OneHotEncoder(categories=[data]*1,sparse=False,
                           handle_unknown = "ignore")
    # transform data
    aa = encoder.fit_transform(IGBP.reshape(IGBP.shape[0]*IGBP.shape[1],1))
    # assign 23-D IGBP into 23 columns
    for i in range(1,12,1):
        IGBP_all['IGBP_veg_long'+str(i)] = aa[:,i-1]
    return IGBP_all

## 4) chunk all the input variables

In [44]:
ds_era5land = all_resample#.sel(time=t, method='nearest')#.compute()

ds_ss = xr.Dataset()

ds_ss["Rin"] = Rin.isel(time=slice(0,17000))#.chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})
ds_ss["Rli"] = Rli.isel(time=slice(0,17000))#.chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})
ds_ss["p"] = all_resample["sp"]/100#.chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})   # Pa -> hPa
ds_ss["Ta"] = all_resample["t2m"]- 273.15 #.chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})   # K -> degC
ds_ss["ea"] = vc.calculate_es(all_resample["d2m"] - 273.15)#.chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})
ds_ss["u"] = (all_resample["u10"] ** 2 + all_resample["v10"] ** 2) ** 0.5
# ds_ss["u"] = (all_resample["u10"].chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)}) ** 2 + all_resample["v10"].chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)}) ** 2) ** 0.5
ds_ss["Precip_msr"] = all_resample["tp"]*1000 #.chunk({"latitude": len(hc.latitude), "longitude": len(hc.longitude)})  # mm
ds_ss['LAI'] = lai#.chunk()
ds_ss['CO2'] = ds_co2_10km#.chunk()
ds_ss['SSM'] = SSM#.chunk()

# ds_ss = ds_ss.assign_coords({"longitude": hc.longitude.values, "latitude": hc.latitude.values})
# ds_Vcmo = ds_Vcmo.assign_coords({"longitude": hc.longitude.values, "latitude": hc.latitude.values})
# landcover = landcover.assign_coords({"longitude": hc.longitude.values, "latitude": hc.latitude.values})

ds_ss = ds_ss.isel(time=slice(0,17000))


/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/h

In [45]:
ds_ss['p'].chunks

((17000,), (102, 110, 89), (108, 110, 83))

In [46]:
ds_ss['LAI'].chunks

((17000,), (102, 110, 89), (108, 110, 83))

In [47]:
ds_ss.nbytes/2**30

/home/qiahan/.local/lib/python3.9/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 20 entries) hit.
Flushing oldest 4 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


57.377683378756046

In [54]:
ds_ss = ds_ss.persist()

2023-10-29 15:16:28,386 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 25.01 GiB -- Worker memory limit: 30.00 GiB
2023-10-29 15:16:29,344 - distributed.worker.memory - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 23.07 GiB -- Worker memory limit: 30.00 GiB


In [49]:
ds_ss['p']

<xarray.DataArray 'p' (time: 17000, latitude: 469, longitude: 451)>
dask.array<where, shape=(17000, 469, 451), dtype=float32, chunksize=(17000, 110, 110), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 81.8 81.7 81.6 81.5 ... 35.3 35.2 35.1 35.0
  * longitude  (longitude) float32 2.0 2.1 2.2 2.3 2.4 ... 46.7 46.8 46.9 47.0
  * time       (time) datetime64[ns] 2015-01-01T01:00:00 ... 2015-12-21T04:30:00

## 5) predict fluxes with map_blocks

In [48]:
ds_ss = ds_ss.to_array()

In [49]:
ds_ss = ds_ss.chunk({"time":500, "variable":10})

In [50]:
ds_ss.chunks

((10,),
 (500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500),
 (102, 110, 89),
 (108, 110, 83))

In [51]:
# hc=hc.chunk({"latitude":51, "longitude":51}).squeeze()
hc = hc.squeeze()
ds_Vcmo = ds_Vcmo.chunk({"latitude":ds_ss.chunks[2], "longitude":ds_ss.chunks[3]}).squeeze()
landcover = landcover.chunk({"latitude":ds_ss.chunks[2], "longitude":ds_ss.chunks[3]}).squeeze()

In [52]:
landcover.chunks

((102, 110, 89), (108, 110, 83))

In [53]:
chunks = [ds_ss.chunksizes[v] for v in ['time', 'latitude', 'longitude']]
chunks.append((7,))

In [54]:
estimated_LEH = xr.DataArray(
    name = 'LEH',
    data=dask.array.zeros((len(ds_ss.time), len(ds_ss.latitude), len(ds_ss.longitude),7), chunks=chunks),
    dims=("time", "latitude", "longitude","variable_out"),
    coords={
        "variable_out":range(7), 
        "time":ds_ss.time, 
        "latitude":ds_ss.latitude,
        "longitude":ds_ss.longitude
    }
)

In [55]:
def predictFlux(ds_ss, hc, ds_Vcmo, landcover, IGBP_table, training_testing_append, path_model):
    # ds_ss = ds_ss.reshape(10,-1)
    # time = ds_ss.time
    # ds_ss = ds_ss.squeeze("time")
    # hc = hc.sel(latitude=ds_ss.latitude, longitude=ds_ss.longitude)
    hc_line = np.tile(hc.to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size) #如果和动态变量一样放进ds_ss里显示为一些离散的点，不知道为什么，所以静态变量放在时间循环外面
    print(hc_line.shape)
    # ds_Vcmo = ds_Vcmo.sel(latitude=ds_ss.latitude, longitude=ds_ss.longitude)
    Vcmo_line = np.tile(ds_Vcmo.to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    # landcover = landcover.sel(latitude=ds_ss.latitude, longitude=ds_ss.longitude)
    IGBP_all = landcover_to_igbp(landcover, IGBP_table, training_testing_append)
    IGBP1 = np.tile(IGBP_all.iloc[:,0].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP2 = np.tile(IGBP_all.iloc[:,1].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP3 = np.tile(IGBP_all.iloc[:,2].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP4 = np.tile(IGBP_all.iloc[:,3].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP5 = np.tile(IGBP_all.iloc[:,4].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP6 = np.tile(IGBP_all.iloc[:,5].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP7 = np.tile(IGBP_all.iloc[:,6].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP8 = np.tile(IGBP_all.iloc[:,7].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP9 = np.tile(IGBP_all.iloc[:,8].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP10 = np.tile(IGBP_all.iloc[:,9].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    IGBP11 = np.tile(IGBP_all.iloc[:,10].to_numpy().reshape(1, hc.latitude.size*hc.longitude.size),(ds_ss.time.size,1)).reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    
    
    Rin_line = ds_ss[0,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    Rli_line = ds_ss[1,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    p_line = ds_ss[2,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    Ta_line = ds_ss[3,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    ea_line = ds_ss[4,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    u_line = ds_ss[5,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    Precip_msr_line = ds_ss[6,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    LAI_line = ds_ss[7,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    CO2_line = ds_ss[8,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)
    SSM_line = ds_ss[9,:,:,:].to_numpy().reshape(1, ds_ss.latitude.size*ds_ss.longitude.size*ds_ss.time.size)

    features_arr = np.concatenate((Rin_line, Rli_line, p_line, Ta_line,ea_line, u_line, Precip_msr_line,
                                   LAI_line, hc_line, CO2_line, SSM_line, Vcmo_line,
                                   IGBP1, IGBP2, IGBP3,IGBP4,IGBP5,IGBP6,IGBP7,IGBP8,IGBP9,IGBP10,IGBP11))
    features_arr = features_arr.transpose()
    # Nan value.
    df_features = pd.DataFrame(data=features_arr)
    df_features_drop_nan = df_features.dropna()
    invalid_index = sorted(set(df_features.index.to_list()) - set(df_features_drop_nan.index.to_list()))

    # # Convert the nan value as 0 for the calculation
    where_are_NaNs = np.isnan(features_arr)
    features_arr[where_are_NaNs] = 0
    
    model = load_model(path_model)
    
    # model = path_model
    features_arr1 = pd.DataFrame(data=features_arr)
    features_arr1.columns = ['Rin', 'Rli', 'p', 'Ta', 'ea', 'u', 'Precip_msr','LAI','hc','CO2','SSM','Vcmo',
            'IGBP_veg_long1','IGBP_veg_long2','IGBP_veg_long3','IGBP_veg_long4','IGBP_veg_long5','IGBP_veg_long6','IGBP_veg_long7','IGBP_veg_long8',
            'IGBP_veg_long9','IGBP_veg_long10','IGBP_veg_long11']
    estimated_LEH = model.predict(features_arr1)
    estimated_LEH[invalid_index] = np.nan
    LEH_map = estimated_LEH.reshape(ds_ss.time.size, ds_ss.latitude.size, ds_ss.longitude.size,7)
    
    return xr.DataArray(
        name = 'LEH',
        data=LEH_map,
        dims=("time", "latitude", "longitude","variable_out"),
        coords={
            "variable_out":range(7), 
            "time":ds_ss.time, 
            "latitude":ds_ss.latitude,
            "longitude":ds_ss.longitude
        }
    )
result = xr.map_blocks(
    predictFlux,
    ds_ss,
    args=[hc, ds_Vcmo, landcover],
    kwargs={"IGBP_table":IGBP_table, "training_testing_append":data, "path_model":workingPath+'3RF_train/RFLEH-LAI-hc-CO2-SSM-Vcmo-IGBP_multi7_1core_snellius0629.pkl'},
    template=estimated_LEH,
)

In [56]:
result.chunks

((500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500,
  500),
 (102, 110, 89),
 (108, 110, 83),
 (7,))

In [57]:
result = result.chunk({"latitude":200, "longitude":200})

In [58]:
result = result.assign_coords(variable_out=['LEtot','Htot','Rntot', 'RSSM', 'SIF685', 'SIF740', 'Actot'])
result_ds = result.to_dataset(dim="variable_out")

In [59]:
result_ds.chunks

/home/qiahan/.local/lib/python3.9/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 20 entries) hit.
Flushing oldest 4 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


Frozen({'time': (500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500), 'latitude': (200, 101), 'longitude': (200, 101)})

In [60]:
%%time
result_ds.to_zarr("/gpfs/work2/0/ttse0619/qianqian/global_data_Qianqian/5output_data/2014fluxes_30degrees.zarr", mode='w')

/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/qiahan/.conda/envs/firstEnv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse

CPU times: user 3min 12s, sys: 17.6 s, total: 3min 30s
Wall time: 12min 6s


In [ ]:
result = result.persist()

In [ ]:
result[2000,:,:,1].plot()

In [ ]:
start_time = time.time()  # Record the start time
result1 = result.compute()
end_time = time.time()  # Record the end time
total_running_time = end_time - start_time
print("Total Running Time:", total_running_time)

In [ ]:
result1

In [ ]:
result1[2000,:,:,1].plot()#.sel(longitude=slice(2,17),latitude=slice(55,40)).plot() 

In [ ]:
result1[2000,:,:,1].plot() 

In [ ]:
np.nanmean(np.nanmean(result1[:,:,:,1],axis=(1,2)))

In [ ]:
result1[:,:,:,1]

## export the result